In [8]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    api_key=os.environ['GROQ_API_KEY']
)

response=llm.invoke("City of rangoon")
response.content

'The City of Rangoon, also known as Yangon, is the largest city in Myanmar (formerly Burma). It was the country\'s capital until 2006, when the government moved the capital to Naypyidaw. Rangoon is still the country\'s commercial and cultural center.\n\nHere are some key facts about the City of Rangoon:\n\n1. **History**: Rangoon was founded in 1755 by King Alaungpaya, the founder of the Konbaung Dynasty. The city was originally called Dagon, but it was renamed Yangon (meaning "end of strife") in 1755.\n2. **Colonial era**: In 1885, the British annexed Burma and made Rangoon the capital of the colony. During this period, the city underwent significant modernization and development, with the construction of roads, railways, and buildings.\n3. **Architecture**: Rangoon is known for its mix of colonial and traditional Burmese architecture. The city is home to many historic buildings, including the Shwedagon Pagoda, the Sule Pagoda, and the High Court.\n4. **Shwedagon Pagoda**: This 99-met

In [9]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jobs.nike.com/job/R-40715?from=job%20search%20funnel")
page_data=loader.load().pop().page_content
page_data

USER_AGENT environment variable not set, consider setting it to identify your requests.


'Apply for Software Engineer II, ITC\n\nSearch JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer II, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc. 

In [12]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)
chain_extract=prompt_extract|llm
res=chain_extract.invoke(input={'page_data':page_data})
print(res.content)


```json
{
  "role": "Software Engineer II, ITC",
  "experience": "2-5 years",
  "skills": [
    "Data engineering",
    "Python or Java",
    "SQL",
    "Data Bricks Airflow, EMR, Hive, Spark",
    "Microservices architecture",
    "Domain driven design",
    "RESTful API's",
    "Data management fundamentals",
    "Data storage principles",
    "CI/CD",
    "DevOps",
    "Cloud services (AWS - EC2, ECS, API gateway, Lambda)",
    "Marketing technology experience (plus)",
    "Adobe toolset experience (plus)"
  ],
  "description": "Design, build, and maintain large datasets and data pipelines to support building out our new Customer Data Platform (CDP). Innovate, architect, design, and implement highly available, scalable systems, focusing on highly visible marketing initiatives. Collaborate with upstream and downstream technical teams to develop integrations to provide a seamless flow of our marketing assets from planning to delivery."
}
```


In [13]:
from langchain_core.output_parsers import JsonOutputParser
json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'role': 'Software Engineer II, ITC',
 'experience': '2-5 years',
 'skills': ['Data engineering',
  'Python or Java',
  'SQL',
  'Data Bricks Airflow, EMR, Hive, Spark',
  'Microservices architecture',
  'Domain driven design',
  "RESTful API's",
  'Data management fundamentals',
  'Data storage principles',
  'CI/CD',
  'DevOps',
  'Cloud services (AWS - EC2, ECS, API gateway, Lambda)',
  'Marketing technology experience (plus)',
  'Adobe toolset experience (plus)'],
 'description': 'Design, build, and maintain large datasets and data pipelines to support building out our new Customer Data Platform (CDP). Innovate, architect, design, and implement highly available, scalable systems, focusing on highly visible marketing initiatives. Collaborate with upstream and downstream technical teams to develop integrations to provide a seamless flow of our marketing assets from planning to delivery.'}

In [ ]:
import pandas as pd
df=pd.read_csv('./my_portfolio.csv')
df

In [15]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [19]:
#testing
links = collection.query(query_texts=['tv'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/android-portfolio'}]]

In [ ]:
job=json_res
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

In [24]:
prompt_email = PromptTemplate.from_template(
   """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
)
chain_email=prompt_email|llm
res=chain_email.invoke({'job_description':str(job),'link_list':links})
print(res.content)

Subject: Expert Software Engineering Services for Your Customer Data Platform (CDP)

Dear Hiring Manager,

I came across your job posting for a Software Engineer II, ITC, and I'm excited to introduce you to AtliQ, a leading AI & Software Consulting company. Our team of experts specializes in designing, building, and maintaining large datasets and data pipelines, which aligns perfectly with your requirements.

With 2-5 years of experience in data engineering, our engineers are well-versed in Python, Java, SQL, and various data management tools like Data Bricks Airflow, EMR, Hive, and Spark. We also have expertise in microservices architecture, domain-driven design, RESTful APIs, and CI/CD pipelines. Our experience with cloud services, particularly AWS (EC2, ECS, API Gateway, Lambda), will ensure seamless integration with your existing infrastructure.

Our portfolio showcases our capabilities in:

* Data Engineering: https://example.com/ml-python-portfolio
* DevOps: https://example.com/d